In [9]:
# import itertools
# import os
# import numpy as np
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import glob
# import json
# import logging
# import sys
# import torch.nn as nn
# import torch.optim as optim
# import random
# def batch_equalizer_fn(args):
#     eeg = args[0]
# #     print("eegshape=",eeg.shape)
#     num_stimuli = len(args) - 1
#     # repeat eeg num_stimuli times
#     new_eeg = torch.cat([eeg] * num_stimuli, dim=0)
#     all_features = [new_eeg]
# #     print("all_features=",all_features[0].shape)

#     # create args
#     args_to_zip = [args[i::num_stimuli] for i in range(1, num_stimuli + 1)]
# #     print("args_to_zip=",args_to_zip[0].shape,args_to_zip[1].shape)

#     for stimuli_features in zip(*args_to_zip):
# #         print("stimuli_features=",stimuli_features[0].shape,stimuli_features[1].shape)
#         for i in range(num_stimuli):
#             shift = i
#             shifted_tuple = tuple(stimuli_features[(j - shift) % len(stimuli_features)] for j in range(len(stimuli_features)))
#             stimulus_rolled = torch.stack(shifted_tuple)
# #             print("stimulus_rolled=", stimulus_rolled.shape)
#             # reshape stimulus_rolled to merge the first two dimensions
#             stimulus_rolled = stimulus_rolled.view(-1, stimulus_rolled.size(2), stimulus_rolled.size(3))
# #             print("stimulus_rolled1=", stimulus_rolled.shape)

#             all_features.append(stimulus_rolled)
# #     print("all_features1=",all_features)
    
#     labels_list = [torch.tensor([[1 if ii == i else 0 for ii in range(num_stimuli)]]) for i in range(num_stimuli)]
#     labels = torch.cat([label.repeat(eeg.size(0), 1) for label in labels_list], dim=0)
# #     print("labels=",labels)
# #     print("tuple(all_features)=", tuple(all_features))

#     return tuple(all_features), labels

# def shuffle_fn(args, number_mismatch):
#     # repeat the last argument number_mismatch times
#     args = list(args)
#     for _ in range(number_mismatch):
#         args.append(args[-1][torch.randperm(args[-1].size(0))])
#     return tuple(args)

# # Function to create frames from a tensor
# def frame_tensor(tensor, window_length, hop_length):
#     num_frames = (tensor.size(0) - window_length) // hop_length + 1
#     frames = torch.stack(
#         [tensor[i * hop_length : i * hop_length + window_length] for i in range(num_frames)]
#     )
#     return frames

# def process_eeg(original_tensors_list):
#     reshaped_tensors = [tensor[i].view(320, 64) for tensor in original_tensors_list for i in range(tensor.size(0))]

#     lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
#     lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

#     # Shuffle the lists
#     random.shuffle(lists_of_tensors)

#     final_tensors = []
#     for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
#         concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 64)
#         final_tensors.append(concatenated_tensors)

#     return final_tensors
# def process_stimuli(original_tensors_list):
#     reshaped_tensors = [tensor[i].view(320, 1) for tensor in original_tensors_list for i in range(tensor.size(0))]

#     lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
#     lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

#     # Shuffle the lists
#     random.shuffle(lists_of_tensors)

#     final_tensors = []
#     for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
#         concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 1)
#         final_tensors.append(concatenated_tensors)

#     return final_tensors
# class PyTorchDataGenerator(Dataset):
#     def __init__(self, files, window_length):
#         self.window_length = window_length
#         self.files = self.group_recordings(files)

#     def group_recordings(self, files):
#         new_files = []
#         grouped = itertools.groupby(
#             sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3])
#         )
#         for recording_name, feature_paths in grouped:
#             new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
# #         print("new_files=", new_files)
#         return new_files
    

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, recording_index):
#         data = []
#         for feature in self.files[recording_index]:
#             f = np.load(feature).astype(np.float32)
# #             print("f_before=", f.shape)
#             if f.ndim == 1:
#                 f = f[:, None]
# #                 print("f_after=", f.shape)
#             data += [f]
# #         print("data_before=", data)
#         data = self.prepare_data(data)
# #         print("data_after=", data)
# #         print("tuple(torch.tensor(x) for x in data)=",tuple(torch.tensor(x) for x in data))
#         return tuple(torch.tensor(x) for x in data)

#     def __call__(self):
#         for idx in range(self.__len__()):
#             yield self.__getitem__(idx)

#             if idx == self.__len__() - 1:
#                 self.on_epoch_end()

#     def on_epoch_end(self):
#         np.random.shuffle(self.files)

#     def prepare_data(self, data):
#         # make sure data has dimensionality of (n_samples, n_features)
#         return data

# def create_pytorch_dataset(
#     data_generator,
#     window_length,
#     batch_equalizer_fn=None,
#     frame_tensor=None,
#     process_eeg=None,
#     process_stimuli=None,
#     hop_length=64,
#     batch_size=64,
#     number_mismatch=None,
#     data_types=(torch.float32, torch.float32),
#     feature_dims=(64, 1)
# ):
#     dataset = data_generator
#     if frame_tensor is not None:
#         i=0
#         for data in dataset:
#             dataset = [(frame_tensor(data[0], window_length, hop_length),frame_tensor(data[1], window_length, hop_length),)]


#     if number_mismatch is not None:
#         # map second argument to shifted version
#         dataset = [
#         shuffle_fn(data, number_mismatch) for data in dataset
#     ]
    
#     if process_eeg is not None and process_stimuli is not None:
#         # map second argument to shifted version
#         dataset=[process_eeg([data[0] for data in dataset]),
#         process_stimuli([data[1] for data in dataset]),
#         process_stimuli([data[2] for data in dataset]),
#         process_stimuli([data[3] for data in dataset]),
#         process_stimuli([data[4] for data in dataset]),
#         process_stimuli([data[5] for data in dataset])]
#         dataset = [tuple([dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]]) for i in range(len(dataset[0]))]
# #         print(dataset[0][0].shape,dataset[0][1].shape,dataset[0][2].shape)

#     if batch_equalizer_fn is not None:
#         # Create the labels and make sure classes are balanced
#         dataset = [
#             tuple(batch_equalizer_fn(args)) for args in dataset
#         ]

#     return tuple(dataset)

# window_length_s = 5
# fs = 64

# window_length = window_length_s * fs  # 5 seconds
# # Hop length between two consecutive decision windows
# hop_length = 64

# epochs = 100
# patience = 5
# batch_size = 64 #fixed in the code
# only_evaluate = True
# number_mismatch = 4 # or 4



# training_log_filename = "training_log_{}_{}.csv".format(number_mismatch, window_length_s)
# data_folder = "split_data/split_data"

# # stimulus feature which will be used for training the model. Can be either 'envelope' ( dimension 1) or 'mel' (dimension 28)
# stimulus_features = ["envelope"]
# stimulus_dimension = 1

# # uncomment if you want to train with the mel spectrogram stimulus representation
# # stimulus_features = ["mel"]
# # stimulus_dimension = 10

# features = ["eeg"] + stimulus_features
# # print("features=", features)
# train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# # Create list of numpy array files
# train_generator = PyTorchDataGenerator(train_files, window_length)
# import pdb
# dataset_train = create_pytorch_dataset(train_generator, window_length, batch_equalizer_fn,frame_tensor,process_eeg,process_stimuli,
#                                   hop_length, batch_size,
#                                   number_mismatch=number_mismatch,
#                                   data_types=(torch.float32, torch.float32),
#                                   feature_dims=(64, stimulus_dimension))

# val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# val_generator = PyTorchDataGenerator(val_files, window_length)
# dataset_val = create_pytorch_dataset(val_generator,  window_length, batch_equalizer_fn,frame_tensor,process_eeg,process_stimuli,
#                                   hop_length, batch_size,
#                                   number_mismatch=number_mismatch,
#                                   data_types=(torch.float32, torch.float32),
#                                   feature_dims=(64, stimulus_dimension))



In [1]:
import itertools
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
import json
import logging
import sys
import torch.nn as nn
import torch.optim as optim
import random
def batch_equalizer_fn(args):
    eeg = args[0]
#     print("eegshape=",eeg.shape)
    num_stimuli = len(args) - 1
    # repeat eeg num_stimuli times
    new_eeg = torch.cat([eeg] * num_stimuli, dim=0)
    all_features = [new_eeg]
#     print("all_features=",all_features[0].shape)

    # create args
    args_to_zip = [args[i::num_stimuli] for i in range(1, num_stimuli + 1)]
#     print("args_to_zip=",args_to_zip[0].shape,args_to_zip[1].shape)

    for stimuli_features in zip(*args_to_zip):
#         print("stimuli_features=",stimuli_features[0].shape,stimuli_features[1].shape)
        for i in range(num_stimuli):
            shift = i
            shifted_tuple = tuple(stimuli_features[(j - shift) % len(stimuli_features)] for j in range(len(stimuli_features)))
#             print(len(shifted_tuple))
            stimulus_rolled = torch.stack(shifted_tuple)
#             print("stimulus_rolled=", stimulus_rolled.shape)
            # reshape stimulus_rolled to merge the first two dimensions
            stimulus_rolled = stimulus_rolled.view(-1, stimulus_rolled.size(2), stimulus_rolled.size(3))
#             print("stimulus_rolled1=", stimulus_rolled.shape)

            all_features.append(stimulus_rolled)
#     print("all_features1=",all_features)
    
    labels_list = [torch.tensor([[1 if ii == i else 0 for ii in range(num_stimuli)]]) for i in range(num_stimuli)]
    labels = torch.cat([label.repeat(eeg.size(0), 1) for label in labels_list], dim=0)
#     print("labels=",labels)
#     print("tuple(all_features)=", tuple(all_features))

    return tuple(all_features), labels

def shuffle_fn(args, number_mismatch):
    # repeat the last argument number_mismatch times
    args = list(args)
    for _ in range(number_mismatch):
        args.append(args[-1][torch.randperm(args[-1].size(0))])
    return tuple(args)

# Function to create frames from a tensor
def frame_tensor(tensor, window_length, hop_length):
    num_frames = (tensor.size(0) - window_length) // hop_length + 1
    frames = torch.stack(
        [tensor[i * hop_length : i * hop_length + window_length] for i in range(num_frames)]
    )
    return frames

def process_eeg(original_tensors_list):
    reshaped_tensors = [tensor[i].view(320, 64) for tensor in original_tensors_list for i in range(tensor.size(0))]

    lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
    lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

    # Shuffle the lists
    random.shuffle(lists_of_tensors)

    final_tensors = []
    for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
        concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 64)
        final_tensors.append(concatenated_tensors)

    return final_tensors
def process_stimuli(original_tensors_list):
    reshaped_tensors = [tensor[i].view(320, 10) for tensor in original_tensors_list for i in range(tensor.size(0))]

    lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
    lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

    # Shuffle the lists
    random.shuffle(lists_of_tensors)

    final_tensors = []
    for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
        concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 10)
        final_tensors.append(concatenated_tensors)

    return final_tensors
class PyTorchDataGenerator(Dataset):
    def __init__(self, files, window_length):
        self.window_length = window_length
        self.files = self.group_recordings(files)

    def group_recordings(self, files):
        new_files = []
        grouped = itertools.groupby(
            sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3])
        )
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
#         print("new_files=", new_files[0:2])
        return new_files
    

    def __len__(self):
        return len(self.files)

    def __getitem__(self, recording_index):
        data = []
        for feature in self.files[recording_index]:
            f = np.load(feature).astype(np.float32)
#             print("f_before=", f.shape)
            if f.ndim == 1:
                f = f[:, None]
#                 print("f_after=", f.shape)
            data += [f]
#         print("data_before=", data)
        data = self.prepare_data(data)
#         print("data_after=", data)
#         print("tuple(torch.tensor(x) for x in data)=",tuple(torch.tensor(x) for x in data))
        return tuple(torch.tensor(x) for x in data)

    def __call__(self):
        for idx in range(self.__len__()):
            yield self.__getitem__(idx)

            if idx == self.__len__() - 1:
                self.on_epoch_end()

    def on_epoch_end(self):
        np.random.shuffle(self.files)

    def prepare_data(self, data):
        # make sure data has dimensionality of (n_samples, n_features)
        return data

def create_pytorch_dataset(
    data_generator,
    window_length,
    batch_equalizer_fn=None,
    frame_tensor=None,
    process_eeg=None,
    process_stimuli=None,
    hop_length=64,
    batch_size=64,
    number_mismatch=None,
    data_types=(torch.float32, torch.float32),
    feature_dims=(64, 1)
):
    dataset = data_generator
    if frame_tensor is not None:
        i=0
        for data in dataset:
            dataset = [(frame_tensor(data[0], window_length, hop_length),frame_tensor(data[1], window_length, hop_length))]


    if number_mismatch is not None:
        # map second argument to shifted version
        dataset = [
        shuffle_fn(data, number_mismatch) for data in dataset
    ]
    
    if process_eeg is not None and process_stimuli is not None:
        # map second argument to shifted version
        dataset=[process_eeg([data[0] for data in dataset]),
        process_stimuli([data[1] for data in dataset])]
        dataset = [tuple([dataset[0][i],dataset[1][i]]) for i in range(len(dataset[0]))]
#         print(dataset[0][0].shape,dataset[0][1].shape,dataset[0][2].shape)

    if batch_equalizer_fn is not None:
        # Create the labels and make sure classes are balanced
        dataset = [
            tuple(batch_equalizer_fn(args)) for args in dataset
        ]

    return tuple(dataset)

window_length_s = 5
fs = 64

window_length = window_length_s * fs  # 5 seconds
# Hop length between two consecutive decision windows
hop_length = 64

epochs = 100
patience = 5
batch_size = 64 #fixed in the code
only_evaluate = True
number_mismatch = 4 # or 4



training_log_filename = "training_log_{}_{}.csv".format(number_mismatch, window_length_s)
data_folder = "split_data/split_data"

# stimulus feature which will be used for training the model. Can be either 'envelope' ( dimension 1) or 'mel' (dimension 28)
stimulus_features = ["mel"]
stimulus_dimension = 10

# uncomment if you want to train with the mel spectrogram stimulus representation
# stimulus_features = ["mel"]
# stimulus_dimension = 10

features = ["eeg"] + stimulus_features
# print("features=", features)
train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# Create list of numpy array files
train_generator = PyTorchDataGenerator(train_files, window_length)
import pdb
dataset_train = create_pytorch_dataset(train_generator, window_length, None,frame_tensor,process_eeg,process_stimuli,
                                  hop_length, batch_size,
                                  number_mismatch=None,
                                  data_types=(torch.float32, torch.float32),
                                  feature_dims=(64, stimulus_dimension))

val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
val_generator = PyTorchDataGenerator(val_files, window_length)
dataset_val = create_pytorch_dataset(val_generator,  window_length, None,frame_tensor,process_eeg,process_stimuli,
                                  hop_length, batch_size,
                                  number_mismatch=None,
                                  data_types=(torch.float32, torch.float32),
                                  feature_dims=(64, stimulus_dimension))



In [13]:
c = 0 
for i in dataset_train:
    print(i[0].shape)
    c += 1
    if c == 2:
        break

torch.Size([64, 320, 64])
torch.Size([64, 320, 64])


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm 

class ResNet34EEG(nn.Module):
    def __init__(self):
        super(ResNet34EEG, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.conv1 = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        return self.model(x)

class ResNet34Mel(nn.Module):
    def __init__(self):
        super(ResNet34Mel, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.conv1 = nn.Conv2d(10, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        return self.model(x)


device = 'cuda'

resnet_eeg = ResNet34EEG().to(device)
resnet_mel = ResNet34Mel().to(device)


In [29]:
optimizer = optim.Adam(list(resnet_eeg.parameters()) + list(resnet_mel.parameters()), lr=0.001)
batch_size = 2
num_epochs = 1000
for epoch in range(num_epochs):
    total_loss = 0.0
    for eeg_input, mel_input in tqdm(dataset_train, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        # print(eeg_input.shape)
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        output_eeg = resnet_eeg(eeg_input)
        output_mel = resnet_mel(mel_input)

        loss = torch.abs(output_eeg - output_mel).sum()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

# Save the trained models if needed
torch.save(resnet_eeg.state_dict(), 'resnet_eeg.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel.pth')

Epoch 1/1000: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.44it/s]


Epoch 1/1000, Average Loss: 98.72620544433593


Epoch 2/1000: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.11it/s]


Epoch 2/1000, Average Loss: 56.95190620422363


Epoch 3/1000: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.43it/s]


...

Epoch 63/1000, Average Loss: 14.622556781768798


Epoch 64/1000: 100%|███████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.27it/s]


Epoch 64/1000, Average Loss: 14.102963066101074


Epoch 65/1000:  80%|██████████████████████████████████████████████████████▍             | 8/10 [00:00<00:00, 10.94it/s]


KeyboardInterrupt: 

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm  

class CustomResNet(nn.Module):
    def __init__(self, input_channels):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

resnet_eeg_soft = CustomResNet(64)
resnet_mel_soft = CustomResNet(10)

resnet_eeg_soft = resnet_eeg_soft.to(device)
resnet_mel_soft = resnet_mel_soft.to(device)


In [50]:
optimizer = optim.Adam(list(resnet_eeg_soft.parameters()) + list(resnet_mel_soft.parameters()), lr=0.001)

batch_size = 8
num_epochs = 100

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    total_loss = 0.0
    for eeg_input, mel_input in tqdm(dataset_train, desc=f'Epoch {epoch+1}/{num_epochs}'):
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        optimizer.zero_grad()

        output_eeg = resnet_eeg_soft(eeg_input)
        output_mel = resnet_mel_soft(mel_input)

        # loss = torch.abs(output_eeg - output_mel).sum()
        loss = criterion(output_eeg,output_mel)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

torch.save(resnet_eeg.state_dict(), 'resnet_eeg_soft.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel_soft.pth')

Epoch 1/100: 100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.62it/s]


Epoch 1/100, Average Loss: 0.015461856918409466


Epoch 2/100: 100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.17it/s]


Epoch 2/100, Average Loss: 0.004576381435617805


Epoch 3/100: 100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.03it/s]


...

Epoch 12/100, Average Loss: 0.0021095211151987312


Epoch 13/100: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.71it/s]


Epoch 13/100, Average Loss: 0.0031775612442288548


Epoch 14/100:  90%|██████████████████████████████████████████████████████████████       | 9/10 [00:00<00:00, 12.01it/s]


KeyboardInterrupt: 